# Ercot Summary Page - Calculations (LZ_SOUTH)

## Load in Data

In [17]:
import xlrd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('ggplot')

# Set file path to Ercot Data
file_path = r"/Users/YoungFreeesh/Visual Studio Code/_Python/Web Scraping/Ercot/MASTER-Ercot.xlsx"

# read all data from "Master Data" tab from "MASTER-Ercot" 
dfMASTER = pd.read_excel(file_path, sheet_name = 'Master Data')

# Convert df to a Date Frame
dfMASTER = pd.DataFrame(dfMASTER) 

# Get Headers of "Master Data"
headers = list(dfMASTER.columns.values) 

# Get Unique Months by creating an Array of the active worksheet names
xls = xlrd.open_workbook(file_path, on_demand=True)
SheetNameArray = xls.sheet_names()
UniqueMonths = SheetNameArray[3:]

### Refine the DataFrame
#Only Take LZ_SOUTH
dfMASTER_LZ_SOUTH =  dfMASTER[['Oper Day', 'Interval Ending', 'LZ_SOUTH']].copy(deep=True)
dfMASTER_LZ_SOUTH['Oper Day'] = pd.to_datetime(dfMASTER_LZ_SOUTH['Oper Day'])

# Reset index to Oper Day
dfMASTER_LZ_SOUTH = dfMASTER_LZ_SOUTH.set_index('Oper Day')

dfMASTER_LZ_SOUTH.head(8)


,Interval Ending,LZ_SOUTH
Oper Day,,
2018-01-01,15,26.02
2018-01-01,30,26.01
2018-01-01,45,25.77
2018-01-01,100,25.83
2018-01-01,115,36.30
2018-01-01,130,27.29
2018-01-01,145,26.72
2018-01-01,200,26.62


## Begin Calculations

### 1) Avg. Daily Price

In [18]:
### Initialize dataframe
daily_avg = pd.DataFrame()

# Resample df & compute mean
daily_avg['Mean'] = dfMASTER_LZ_SOUTH.LZ_SOUTH.resample('D').mean()
daily_avg


,Mean
Oper Day,
2018-01-01,58.379375
2018-01-02,51.962813
2018-01-03,58.552188
2018-01-04,52.459687
2018-01-05,26.634583
2018-01-06,24.315625
2018-01-07,13.323958
2018-01-08,19.086563
2018-01-09,20.852187


### 2) Avg. Price - Power was cut off from 3:15pm to 4:15pm 

Remove Prices from: 1515 to 1615 --> Remove (1530, 1545, 1600, 1615)

(i.e. remove those 4 data points and take the avg again)


In [25]:
# 1530 - 1615
dfMASTER_LZ_SOUTH_Optimized = dfMASTER_LZ_SOUTH[ ( (dfMASTER_LZ_SOUTH['Interval Ending'] <= 1515) | (dfMASTER_LZ_SOUTH['Interval Ending'] >= 1630) ) ].copy(deep=True)     
# 1530 - 1715
dfMASTER_LZ_SOUTH_Optimized2 = dfMASTER_LZ_SOUTH[ ( (dfMASTER_LZ_SOUTH['Interval Ending'] <= 1515) | (dfMASTER_LZ_SOUTH['Interval Ending'] >= 1730) ) ].copy(deep=True)     


#daily_avg_Optimized
### Initialize dataframe
daily_avg_Optimized = pd.DataFrame()

# Resample df & compute mean
daily_avg_Optimized['Mean'] = dfMASTER_LZ_SOUTH_Optimized.LZ_SOUTH.resample('D').mean()
#daily_avg_Optimized


daily_avg_Optimized2 = pd.DataFrame()

# Resample df & compute mean
daily_avg_Optimized2['Mean'] = dfMASTER_LZ_SOUTH_Optimized2.LZ_SOUTH.resample('D').mean()
#daily_avg_Optimized


### 3)Difference between (1) and (2) 


In [26]:
difference = daily_avg - daily_avg_Optimized
difference2 = daily_avg - daily_avg_Optimized2

print(daily_avg.describe())
print(daily_avg_Optimized.describe())
print(daily_avg_Optimized2.describe())
print(difference.describe())
print(difference2.describe())

#difference

             Mean
count  211.000000
mean    33.811806
std     35.564612
min     10.797292
25%     19.715781
50%     25.107604
75%     31.193646
max    332.416146
             Mean
count  211.000000
mean    32.136111
std     33.369877
min     10.727283
25%     19.673478
50%     24.576739
75%     30.092609
max    343.752826
             Mean
count  211.000000
mean    30.290346
std     31.728543
min     10.397841
25%     19.427898
50%     24.127727
75%     29.490398
max    354.943636
             Mean
count  211.000000
mean     1.675695
std      8.349710
min    -11.336680
25%     -0.044194
50%      0.161730
75%      0.537536
max     90.084923
             Mean
count  211.000000
mean     3.521460
std     15.173907
min    -22.527491
25%     -0.117822
50%      0.342027
75%      1.403220
max    111.832794


## Create Summary DataFrame

In [27]:
dfSummary = pd.DataFrame()
dfSummary['Avg Daily Price'] = daily_avg['Mean']
dfSummary['Avg Daily Price - Optimized'] = daily_avg_Optimized['Mean']
dfSummary['Avg Daily Price - Optimized 2'] = daily_avg_Optimized2['Mean']
dfSummary['Difference'] = difference['Mean']
dfSummary['Difference 2'] = difference2['Mean']

print('Avg Daily Price SUM: ', daily_avg['Mean'].sum())
print('Avg Daily Price - Optimized SUM: ', daily_avg_Optimized['Mean'].sum())
print('Avg Daily Price - Optimized 2 SUM: ', daily_avg_Optimized2['Mean'].sum())
print('Difference SUM: ', difference['Mean'].sum())
print('Difference 2 SUM: ', difference2['Mean'].sum())


dfSummary



Avg Daily Price SUM:  7134.29114583
Avg Daily Price - Optimized SUM:  6780.71945652
Avg Daily Price - Optimized 2 SUM:  6391.26306818
Difference SUM:  353.571689312
Difference 2 SUM:  743.028077652


,Avg Daily Price,Avg Daily Price - Optimized,Avg Daily Price - Optimized 2,Difference,Difference 2
Oper Day,,,,,
2018-01-01,58.379375,58.246957,57.932955,0.132418,0.446420
2018-01-02,51.962813,52.459783,52.330682,-0.496970,-0.367869
2018-01-03,58.552188,59.273696,59.970909,-0.721508,-1.418722
2018-01-04,52.459687,53.371957,53.959091,-0.912269,-1.499403
2018-01-05,26.634583,26.836087,26.970795,-0.201504,-0.336212
2018-01-06,24.315625,24.681739,25.062045,-0.366114,-0.746420
2018-01-07,13.323958,13.378478,13.478636,-0.054520,-0.154678
2018-01-08,19.086563,19.114457,19.132841,-0.027894,-0.046278
2018-01-09,20.852187,20.935870,20.998864,-0.083682,-0.146676
